#1. Повнозв'язані нейронні мережі
Візьміть дані, з якими ви працювали в лабораторній №1.  Побудуйте повнозв’язану нейронну мережу
прямого поширення для задачі класифікації.
Навчіть її на тренувальній вибірці, протестуйте на тестовій. Порівняйте результати з алгоритмами з Lab 1.
---

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Завантаження та підготовка даних
url='https://drive.google.com/file/d/1QFzghQ5iBStsVt8ktLIAVFBK6RsppeHp/view?usp=sharing'
url_='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url_)

df = df.drop('Id', axis=1)
X = df.drop('Type of glass', axis=1).values
y = df['Type of glass'].values - 1  # класи 0..6

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

# Ваги класів (ручне створення)
# обчислимо ваги по кількості елементів у train
train_classes, counts = np.unique(y_train.cpu().numpy(), return_counts=True)
class_weights_np = np.ones(7, dtype=np.float32)  # всі класи = 1 по замовчуванню
for cls, cnt in zip(train_classes, counts):
    class_weights_np[cls] = len(y_train) / (len(train_classes) * cnt)

class_weights = torch.tensor(class_weights_np, dtype=torch.float32).to(device)

# Модель
class GlassClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 7)
        )

    def forward(self, x):
        return self.layers(x)

model = GlassClassifier().to(device)

# Loss та оптимізатор
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Тренування
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Оцінка
model.eval()
with torch.no_grad():
    y_pred = torch.argmax(model(X_test), dim=1)
    acc = accuracy_score(y_test.cpu(), y_pred.cpu())
    print(f"\nAccuracy on test set: {acc:.4f}\n")
    print("Classification Report:\n")
    print(classification_report(y_test.cpu(), y_pred.cpu(), zero_division=0))


Epoch 50/500, Loss: 1.4150
Epoch 100/500, Loss: 0.8282
Epoch 150/500, Loss: 0.6528
Epoch 200/500, Loss: 0.5953
Epoch 250/500, Loss: 0.4604
Epoch 300/500, Loss: 0.4522
Epoch 350/500, Loss: 0.3984
Epoch 400/500, Loss: 0.3352
Epoch 450/500, Loss: 0.3010
Epoch 500/500, Loss: 0.2770

Accuracy on test set: 0.7674

Classification Report:

              precision    recall  f1-score   support

           0       0.69      0.64      0.67        14
           1       0.77      0.67      0.71        15
           2       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         3
           5       0.67      1.00      0.80         2
           6       0.86      1.00      0.92         6

    accuracy                           0.77        43
   macro avg       0.79      0.88      0.83        43
weighted avg       0.77      0.77      0.76        43



Повнозв’язана нейромережа випередила SVM, Decision Tree та AdaBoost, але
поступилася kNN (0.79) і RandomForest (0.81).
---
Мережа продемонструвала стабільне зменшення loss протягом навчання.
---
FNN показала конкурентний результат, але для табличних даних класичні алгоритми (RandomForest, kNN) часто працюють краще – що й проявилось у цьому експерименті.
---
---

#2. Згорткові нейронні мережі
а) Побудуйте просту згорткову нейронну мережу
(2–3 convolutional шари + fully connected). Навчіть її на обраному вами датасеті.
---
б) Використайте попередньо натреновану архітектуру (наприклад, ResNet, VGG, MobileNet). Замініть вихідний класифікатор
на новий під ваші класи. Проведіть донавчання моделі на вашому датасеті.
---
Порівняйте результати
(точність, швидкість збіжності, кількость даних).
---

In [ ]:
import os
import random
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import gdown

torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# Завантаження архіву
file_id = "1r_wdQGJLAMXKAYMf3OeEQb_7k9VJ3fp3"
url = f"https://drive.google.com/uc?id={file_id}"

output = "Intel_Image_Classification.zip"
gdown.download(url, output, quiet=False)

!unzip -q Intel_Image_Classification.zip -d ./data/

# Перевіримо структуру
print("Folders in ./data/:", os.listdir("./data"))

train_src = "./data/seg_train/seg_train/"
test_src  = "./data/seg_test/seg_test/"
pred_src  = "./data/seg_pred/seg_pred/"


print("Train classes:", os.listdir(train_src))
print("Test classes:", os.listdir(test_src))
print("Pred sample files:", os.listdir(pred_src)[:10])


# DATASET + DATALOADER
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(train_src, transform=transform)
test_dataset  = datasets.ImageFolder(test_src, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Num train images:", len(train_dataset))
print("Num test images:", len(test_dataset))


# Проста CNN модель
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*18*18, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


model = SimpleCNN(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Навчання CNN
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Тест
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"[CNN {epoch+1}/{epochs}] Loss={total_loss:.3f}, Test Acc={acc*100:.2f}%")


# RESNET18 TRANSFER LEARNING
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet.fc = nn.Linear(resnet.fc.in_features, 6)
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=1e-4)

epochs = 3
for epoch in range(epochs):
    resnet.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = resnet(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # on test
    resnet.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            pred = resnet(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"[ResNet {epoch+1}/{epochs}] Loss={total_loss:.3f}, Test Acc={acc*100:.2f}%")


Device: cuda


Downloading...
From (original): https://drive.google.com/uc?id=1r_wdQGJLAMXKAYMf3OeEQb_7k9VJ3fp3
From (redirected): https://drive.google.com/uc?id=1r_wdQGJLAMXKAYMf3OeEQb_7k9VJ3fp3&confirm=t&uuid=b90e1062-e7dd-4233-96dd-0fcf8c619241
To: /content/Intel_Image_Classification.zip
100%|██████████| 363M/363M [00:06<00:00, 53.9MB/s]


Folders in ./data/: ['seg_train', 'seg_pred', 'seg_test']
Train classes: ['sea', 'street', 'forest', 'buildings', 'mountain', 'glacier']
Test classes: ['sea', 'street', 'forest', 'buildings', 'mountain', 'glacier']
Pred sample files: ['13261.jpg', '13014.jpg', '12920.jpg', '8035.jpg', '22630.jpg', '3580.jpg', '5702.jpg', '11391.jpg', '7269.jpg', '20526.jpg']
Num train images: 14034
Num test images: 3000
[CNN 1/5] Loss=353.371, Test Acc=78.27%
[CNN 2/5] Loss=217.706, Test Acc=82.40%
[CNN 3/5] Loss=154.922, Test Acc=83.33%
[CNN 4/5] Loss=102.363, Test Acc=80.83%
[CNN 5/5] Loss=67.006, Test Acc=82.17%
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


[ResNet 1/3] Loss=134.471, Test Acc=92.27%
[ResNet 2/3] Loss=54.523, Test Acc=93.00%
[ResNet 3/3] Loss=27.124, Test Acc=93.27%


Переднавчена ResNet18 перевершила власну CNN на ≈10%, а також тренувалась набагато швидше та стабільніше.
---
Transfer learning показує свою ефективність, особливо на реальних великих зображеннях.
---
Використання великої попередньо натренованої архітектури дає кращий початковий простір ознак, тому навчання потребує менше даних і часу.
---
---

# 3. Вирішіть задачу класифікації текстів (використайте той же датасет, з яким ви працювали в лабораторній № 2) двома способами:

а) Побудуйте модель з вбудованим Embedding шаром (ініціалізованим випадковими вагами). Використайте RNN / LSTM / GRU для класифікації. Навчіть модель на вашому датасеті.
---
б) Завантажте готові embeddings (наприклад, GloVe). Ініціалізуйте Embedding шар цими вагами.
Проведіть навчання.
---
Порівняйте якість класифікації у (а) та (б). Чи покращилися метрики
при використанні pretrained embeddings? Наскільки швидше/стабільніше відбулося
навчання?
---

In [ ]:
import pandas as pd
import numpy as np
import re, string
from collections import Counter
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import random

# Підготовка середовища
torch.manual_seed(42)
random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Завантажуємо дані
!gdown --id 1IwcM9-Dyir1O2JhOxgnRo3Tb9xAt8lWP -O reviews.txt

# Читаємо пострічково
with open("reviews.txt", "r", encoding="utf-8") as f:
    lines = f.read().splitlines()

# Створюємо DataFrame
import pandas as pd
data = pd.DataFrame(lines, columns=['text'])

# Витягуємо label і текст
data['label'] = data['text'].apply(lambda x: x.split(' ')[0])
data['text']  = data['text'].apply(lambda x: ' '.join(x.split(' ')[1:]))
data['label'] = data['label'].replace({'__label__1': 0, '__label__2': 1})


# Очищення тексту
def clean_text(text):
    text = text.lower()
    text = re.sub(r"@\S+", " ", text)
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

data['clean_text'] = data['text'].apply(clean_text)

# Токенізація (проста, без punkt_tab)
tokens = [t.split() for t in data['clean_text']]

# Створюємо словник
counter = Counter()
for tok_list in tokens:
    counter.update(tok_list)

MAX_VOCAB = 20000
specials = ["[PAD]", "[UNK]"]
most_common = counter.most_common(MAX_VOCAB - len(specials))
itos = specials + [w for w, _ in most_common]   # index -> string
stoi = {w:i for i,w in enumerate(itos)}         # string -> index

PAD_IDX = stoi["[PAD]"]
UNK_IDX = stoi["[UNK]"]

# Кодування та паддінг
MAX_LEN = 100

def encode(tokens):
    return [stoi.get(t, UNK_IDX) for t in tokens]

def pad_sequence(seq):
    seq = seq[:MAX_LEN] + [PAD_IDX] * max(0, MAX_LEN - len(seq))
    return torch.tensor(seq, dtype=torch.long)

encoded_texts = [encode(tok_list) for tok_list in tokens]
X = torch.stack([pad_sequence(seq) for seq in encoded_texts])
y = torch.tensor(data['label'].values, dtype=torch.float32)

# DataLoader
dataset = TensorDataset(X, y)
VAL_FRAC = 0.2
val_sz = int(len(dataset) * VAL_FRAC)
train_sz = len(dataset) - val_sz
train_ds, val_ds = random_split(dataset, [train_sz, val_sz],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = val_loader  # тут використаємо val як тест для прикладу

# Модель BiLSTM
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, 1)

    def forward(self, x):
        x = self.embedding(x)
        out, (h, c) = self.lstm(x)
        h_cat = torch.cat([h[-2], h[-1]], dim=1)
        logits = self.fc(h_cat)
        return logits.squeeze(1)

# Функції навчання та оцінки
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_acc, n = 0.0, 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        preds = (torch.sigmoid(logits) >= 0.5).long()
        total_loss += loss.item() * xb.size(0)
        total_acc  += (preds == yb.long()).sum().item()
        n += xb.size(0)
    return total_loss/n, total_acc/n

# Навчання без pretrained embeddings
vocab_size = len(stoi)
emb_dim = 100
hidden_dim = 64

model_basic = BiLSTM(vocab_size, emb_dim, hidden_dim, PAD_IDX).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_basic.parameters(), lr=1e-3)

n_epochs = 5
for epoch in range(n_epochs):
    train_loss = train_one_epoch(model_basic, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model_basic, val_loader, criterion, device)
    print(f"[Basic] Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc*100:.2f}%")

# Навчання з GloVe
# Завантажуємо GloVe
!wget -q http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

def load_glove_txt(path):
    vectors = {}
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            parts = line.rstrip().split(" ")
            word = parts[0]
            vec = np.asarray(parts[1:], dtype=np.float32)
            vectors[word] = vec
    dim = len(next(iter(vectors.values())))
    return vectors, dim

glove, emb_dim = load_glove_txt("glove.6B.100d.txt")

# Збираємо embedding matrix
emb_matrix = np.random.normal(scale=0.1, size=(vocab_size, emb_dim)).astype(np.float32)
emb_matrix[PAD_IDX] = 0.0
hit = 0
for w, idx in stoi.items():
    v = glove.get(w)
    if v is not None:
        emb_matrix[idx] = v
        hit += 1
print(f"GloVe coverage: {hit}/{vocab_size} = {hit/vocab_size:.1%}")

pretrained_emb = nn.Embedding.from_pretrained(torch.tensor(emb_matrix),
                                              freeze=True,
                                              padding_idx=PAD_IDX).to(device)

model_glove = BiLSTM(vocab_size, emb_dim, hidden_dim, PAD_IDX).to(device)
model_glove.embedding = pretrained_emb

optimizer = torch.optim.Adam(model_glove.parameters(), lr=1e-3)

for epoch in range(n_epochs):
    train_loss = train_one_epoch(model_glove, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model_glove, val_loader, criterion, device)
    print(f"[GloVe] Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc*100:.2f}%")


Device: cuda
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1IwcM9-Dyir1O2JhOxgnRo3Tb9xAt8lWP
From (redirected): https://drive.google.com/uc?id=1IwcM9-Dyir1O2JhOxgnRo3Tb9xAt8lWP&confirm=t&uuid=8f00f3b3-0559-461d-9a1f-4b9642a8be53
To: /content/reviews.txt
100% 177M/177M [00:05<00:00, 35.3MB/s]


/tmp/ipython-input-3336925858.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['label'] = data['label'].replace({'__label__1': 0, '__label__2': 1})


[Basic] Epoch 1: train_loss=0.2909, val_loss=0.2200, val_acc=91.30%
[Basic] Epoch 2: train_loss=0.1905, val_loss=0.1960, val_acc=92.46%
[Basic] Epoch 3: train_loss=0.1597, val_loss=0.1908, val_acc=92.57%
[Basic] Epoch 4: train_loss=0.1369, val_loss=0.1919, val_acc=92.84%
[Basic] Epoch 5: train_loss=0.1175, val_loss=0.1956, val_acc=92.81%
GloVe coverage: 19746/20000 = 98.7%
[GloVe] Epoch 1: train_loss=0.3274, val_loss=0.2951, val_acc=87.38%
[GloVe] Epoch 2: train_loss=0.2377, val_loss=0.2217, val_acc=91.04%
[GloVe] Epoch 3: train_loss=0.2087, val_loss=0.2108, val_acc=91.63%
[GloVe] Epoch 4: train_loss=0.1922, val_loss=0.1957, val_acc=92.28%
[GloVe] Epoch 5: train_loss=0.1797, val_loss=0.1949, val_acc=92.32%


У цьому конкретному датасеті власні Embedding показали трохи кращу метрику (≈ +0.5%).
---
GloVe дав:
---
 - кращу семантичну ініціалізацію
 - плавніше навчання після 2–3 епох
 - але стартував гірше, тому фінальний результат був трохи нижчий.

Це типова ситуація, коли великий датасет + проста структура => модель сама навчає оптимальні embeddings.
---
Різниця мінімальна, обидві моделі працюють дуже добре (>92%).
---